In [0]:
import pandas as pd
import numpy as np
from functools import reduce

In [2]:
alunos = pd.read_csv('alunos_caxias_2018.csv')
alunos.head()

,ID_ALUNO,NU_ANO_CENSO,CO_IES,TP_GRAU_ACADEMICO,TP_MODALIDADE_ENSINO,NO_CURSO,NU_IDADE,ANO_INGRESSO,TP_SITUACAO,TP_NIVEL_ACADEMICO,TP_SEXO
0,71CCAB7D5BAE6603B3995CA2EB8F9534,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,34,2014,Cursando,Graduação,Masculino
1,FBC7BF40B3F9AC1C46F78CFF70AC141B,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,28,2018,Cursando,Graduação,Masculino
2,43ABA22A4C7DC2F6E69C71CE90D339A0,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,27,2017,Matrícula trancada,Graduação,Masculino
3,10707CB293B32AD1BC23EB641765BCE7,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,28,2014,Matrícula trancada,Graduação,Feminino
4,7C95F1E0E536524DC3281C4452030F72,2018,Universidade De Caxias Do Sul,Licenciatura,Presencial,História,54,2007,Matrícula trancada,Graduação,Feminino


In [0]:
faixas_idades = [[17],
                [17,18],
                [17,18,19],
                [17,18,19,20],
                [17,18,19,20,21],
                [17,18,19,20,21,22],
                [17,18,19,20,21,22,23],
                [17,18,19,20,21,22,23,24],
                [17,18,19,20,21,22,23,24,25],
                [17,18,19,20,21,22,23,24,25,26],
                [17,18,19,20,21,22,23,24,25,26,27],
                [17,18,19,20,21,22,23,24,25,26,27,28],
                [17,18,19,20,21,22,23,24,25,26,27,28,29],
                [17,18,19,20,21,22,23,24,25,26,27,28,29,30]]

nomes_colunas = ['17',
                '17-18',
                '17-19',
                '17-20',
                '17-21',
                '17-22',
                '17-23',
                '17-24',
                '17-25',
                '17-26',
                '17-27',
                '17-28',
                '17-29',
                '17-30']

In [0]:
situacao = (alunos.TP_SITUACAO == 'Cursando')
nivel = (alunos.TP_NIVEL_ACADEMICO == 'Graduação')
ano_ingresso = (alunos.ANO_INGRESSO == 2018)


dados_faixa_idade = []

for faixa_idade, nome_coluna in zip(faixas_idades, nomes_colunas):
  idades = (alunos.NU_IDADE.isin(faixa_idade))
  ingressantes = alunos.loc[situacao & nivel & ano_ingresso & idades, :].copy()
  contagem = ingressantes.CO_IES.value_counts()
  tabela = pd.DataFrame(zip(contagem.keys(), contagem.values), columns = ['IES', nome_coluna])
  dados_faixa_idade.append(tabela)

In [0]:
siglas = np.load('dicionario_ies_siglas.npy', allow_pickle=True).item()

In [58]:
unir = reduce(lambda left, right: pd.merge(left, right, how = 'outer', on = 'IES'), dados_faixa_idade)
unir.IES.replace(siglas, inplace = True)
unir

,IES,17,17-18,17-19,17-20,17-21,17-22,17-23,17-24,17-25,17-26,17-27,17-28,17-29,17-30
0,UCS,37.0,712,1234,1625,1867,2032,2160,2289,2392,2468,2542,2593,2643,2685
1,FSG,21.0,438,925,1303,1585,1824,2054,2282,2459,2640,2827,2964,3102,3257
2,FTEC,6.0,117,256,368,469,555,668,778,869,940,1035,1117,1195,1259
3,IDEAU,1.0,12,14,20,24,27,32,38,42,46,48,49,51,53
4,Anhanguera,1.0,37,73,112,136,163,184,209,237,251,265,281,293,312
5,FTSG,1.0,7,15,27,35,41,46,57,63,66,72,79,81,85
6,Fátima,NaN,8,23,27,37,41,47,54,60,69,71,71,77,80
7,Murialdo,NaN,7,21,35,43,49,54,56,60,64,69,72,76,79


In [0]:
unir.to_csv('ingressantes_somatorio_idades.csv', index=False)